In [15]:
!pip install tweepy
!pip install textblob
import tweepy
from textblob import TextBlob
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Twitter API credentials
consumer_key = 'put yours'
consumer_secret = 'put yours'
access_token = 'put yours'
access_token_secret = 'put yours'

# Authenticate with Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth)


auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

api = tweepy.API(auth)


In [ ]:
from requests.models import LocationParseError
search_term = 'metaverse security'
since_date = '2023-01-01'
until_date = '2023-04-15'
#location = '39.8283,-98.5795,2500km' # USA geocode
count = 1000


In [ ]:
!pip install emoji
import emoji

In [ ]:
import pandas as pd
import re
!pip install demoji
import demoji

# download demoji's emoji dictionary
demoji.download_codes()

# create empty list to store data
tweets_data = []

# loop through each tweet and extract relevant information
for tweet in tweepy.Cursor(api.search_tweets,
                           q=search_term,
                           #geocode=location,
                           lang='en',
                           #since_id=since_date,
                           count=count).items():
    tweet_text = tweet.text
    tweet_user = tweet.user.screen_name
    tweet_date = tweet.created_at
    #tweet_location = tweet.user.location

    # remove web links
    tweet_text = re.sub(r'http\S+', '', tweet_text)

    # remove usernames and mentions
    tweet_text = re.sub(r'@[^\s]+', '', tweet_text)

    # remove emojis
    emoji_less_text = demoji.replace(tweet_text, "")

    # remove extra white spaces
    tweet_text = re.sub(r'\s+', ' ', emoji_less_text)

    # remove any remaining non-alphanumeric characters and convert to lowercase
    tweet_text = re.sub(r'[^a-zA-Z0-9\s]', '', tweet_text)
    tweet_text = tweet_text.lower()

    # remove non-alphanumeric characters
    alphanumeric_text = re.sub(r'[^a-zA-Z0-9\s]', '', emoji_less_text)

    # append cleaned tweet data to tweets_data list
    tweet_data = [tweet_text, tweet_user, tweet_date] #,tweet_location]
    tweets_data.append(tweet_data)

# create DataFrame from tweets data
df = pd.DataFrame(data=tweets_data, columns=['text', 'user', 'date']) #, 'location'])


In [ ]:
df.head()

In [ ]:
from textblob import TextBlob

# add sentiment polarity and subjectivity columns to the dataframe
df['polarity'] = df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
df['subjectivity'] = df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)

# classify tweets as positive, negative, or neutral based on polarity scores
df['sentiment'] = df['polarity'].apply(lambda score: 'positive' if score > 0 else ('negative' if score < 0 else 'neutral'))


In [ ]:
df.head()

In [ ]:
# count the number of tweets for each sentiment
sentiment_counts = df['sentiment'].value_counts()

# calculate the percentage of tweets for each sentiment
positive_percentage = sentiment_counts['positive'] / len(df) * 100
negative_percentage = sentiment_counts['negative'] / len(df) * 100
neutral_percentage = sentiment_counts['neutral'] / len(df) * 100

print("Positive tweets percentage:", positive_percentage)
print("Negative tweets percentage:", negative_percentage)
print("Neutral tweets percentage:", neutral_percentage)


import matplotlib.pyplot as plt

# plot the sentiment distribution
sentiment_counts.plot.bar()

# add titles and labels to the plot
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Count')

# show the plot
plt.show()


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# define X (features) and y (target) variables
X = df['text']
y = df['sentiment']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# vectorize the text data using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# define the LGB classifier model
lgb_clf = lgb.LGBMClassifier()

# train the model on the training data
lgb_clf.fit(X_train_vec, y_train)

# make predictions on the testing data
y_pred = lgb_clf.predict(X_test_vec)

# calculate accuracy score
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# create a string containing all the tweets
all_tweets = ' '.join(df['text'].tolist())

# generate a wordcloud plot
wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(all_tweets)

# plot the wordcloud
plt.figure(figsize=(8,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


In [ ]:
df.info()

In [ ]:
print(len(df))


In [ ]:
df.to_csv('metaverse_security_crappedtweets.csv', index=False)
